In [1]:
!pip install -U openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/a9/cc/f2bbce0ad52e09cd1aecb724af06385021b42a7317cd5938ba9c8581509d/openai-1.13.3-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/227.4 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/227.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 227.4/227.4 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.12.0
    Uninstalling openai-1.12.0:
      Successfully uninstalled openai-1.12.0



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [111]:
import pandas as pd
import transformers
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import json
from sklearn.metrics import  f1_score,confusion_matrix
from openai import OpenAI
import openai
import json


In [26]:
client = OpenAI(api_key="") # add your key

### Empathetic Dataset

In [145]:
dataset = load_dataset("empathetic_dialogues")
train = pd.DataFrame(dataset['train'])
validation = pd.DataFrame(dataset['validation'])
test = pd.DataFrame(dataset['test'])

In [146]:
# if you want to train faster with less data

train = train[train['context'].isin(['sentimental','surprised'])].sample(60).reset_index(drop = True)
validation = validation[validation['context'].isin(['sentimental','surprised'])].sample(10).reset_index(drop = True)
test = test[test['context'].isin(['sentimental','surprised'])].sample(10).reset_index(drop = True)

train.shape,validation.shape,test.shape

((60, 8), (10, 8), (10, 8))

In [147]:
train['context'].unique(),validation['context'].unique(), test['context'].unique()

(array(['surprised', 'sentimental'], dtype=object),
 array(['sentimental', 'surprised'], dtype=object),
 array(['surprised', 'sentimental'], dtype=object))

In [148]:
train["text"] = train['prompt']+" [SEP] "+ train['utterance']
validation["text"] = validation['prompt']+" [SEP] "+ validation['utterance']
test["text"] = test['prompt']+" [SEP] "+ test['utterance']

In [149]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"context": "' + str(row['context']) + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['text']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


In [150]:
gpt_train = convert_to_gpt35_format(train)
gpt_val = convert_to_gpt35_format(validation)
gpt_test = convert_to_gpt35_format(test)

In [151]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train_em.jsonl"
validation_file_name = "val_em.jsonl"

write_to_jsonl(gpt_train, training_file_name)
write_to_jsonl(gpt_val, validation_file_name)

In [ ]:
# Upload Training and Validation Files
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

# Create Fine-Tuning Job
suffix_name = "gpt_empathi"
response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
    hyperparameters={
    "n_epochs":3
  }
)
response 

FineTuningJob(id='ftjob-7IjxoS60HP3LiB6Il0x28OhM', created_at=1709662898, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-JJq0vmR96TJvz4sOKvRr7ht1', result_files=[], status='validating_files', trained_tokens=None, training_file='file-qg2FBEUQRBYVEHObCKq5IVPY', validation_file='file-NXivCcEzHDQk6U1sYolrVmqO', user_provided_suffix='gpt_empathi')

In [45]:

client.fine_tuning.jobs.list(limit=10)# search for suffix_name

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-7IjxoS60HP3LiB6Il0x28OhM', created_at=1709662898, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-JJq0vmR96TJvz4sOKvRr7ht1', result_files=[], status='validating_files', trained_tokens=None, training_file='file-qg2FBEUQRBYVEHObCKq5IVPY', validation_file='file-NXivCcEzHDQk6U1sYolrVmqO', user_provided_suffix='gpt_empathi')], object='list', has_more=False)

In [155]:
response = client.fine_tuning.jobs.retrieve("ftjob-7IjxoS60HP3LiB6Il0x28OhM") # retrieve based on yours job
response 
     

FineTuningJob(id='ftjob-7IjxoS60HP3LiB6Il0x28OhM', created_at=1709662898, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:gpt-empathi:8zTrwilN', finished_at=1709663515, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-JJq0vmR96TJvz4sOKvRr7ht1', result_files=['file-w15qxZRi9s6DEUkPBwKkfa8x'], status='succeeded', trained_tokens=10221, training_file='file-qg2FBEUQRBYVEHObCKq5IVPY', validation_file='file-NXivCcEzHDQk6U1sYolrVmqO', user_provided_suffix='gpt_empathi')

In [156]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:gpt-empathi:8zTrwilN


In [157]:
def format_test(row):
    formatted_message = [{"role": "user", "content": row['text']}]
    return formatted_message

def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )
    return response.choices[0].message.content

def store_predictions(test_df,y_test, fine_tuned_model_id):
    test_df['Prediction'] = None
    pred = []
    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        pred.append(prediction_result)
    pred = [json.loads(i.replace("'", "\""))['context'] for i in pred]
    print("F1 Score : ", f1_score(pred,y_test,average='weighted'))
    print("Confusion Matrix  : ", confusion_matrix(pred,y_test))
    # test_df.to_csv("predictions.csv")

In [158]:
store_predictions(test,test['context'], fine_tuned_model_id)

F1 Score :  0.9032967032967033
Confusion Matrix  :  [[3 0]
 [1 6]]


### conv_ai_2

In [64]:
dataset = load_dataset("conv_ai_2")
df = pd.DataFrame(dataset['train'])

In [65]:
df = df.sample(100)

In [66]:
def preprocess_dialog(dialog):
    try:
        dialog_list = dialog
        dialog_text = " ".join([d['text'] for d in dialog_list])
        return dialog_text
    except:
        return ""

def preprocess_profile(profile):
    try:
        profile_list = profile
        profile_text = ' '.join([''.join(char_list) for char_list in profile_list])
        return profile_text
    except:
        return ""

# Apply preprocessing
df['dialog_text'] = df['dialog'].apply(preprocess_dialog)
df['bot_profile_text'] = df['bot_profile'].apply(preprocess_profile)
df['user_profile_text'] = df['user_profile'].apply(preprocess_profile)

# Combine dialog and profiles
df['combined_text'] = df['bot_profile_text'] + " [SEP] " + df['user_profile_text'] + " [SEP] " + df['dialog_text']

# Display the processed data
df[['combined_text', 'profile_match']].head()

,combined_text,profile_match
2847,i own a house in florida. i work in it and hav...,1
2611,i do not have a smartphone. i can sew my own c...,1
426,my main transportation is my motorbike. its re...,0
3124,spiderman is my favorite. l also love comic bo...,-1
677,i love to play pobox. i am a 12 year old girl....,0


In [67]:
df = df[['combined_text', 'profile_match']]

In [69]:
df['profile_match'].value_counts()

profile_match
 1    60
 0    29
-1    11
Name: count, dtype: int64

In [70]:
# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    df,
    test_size=0.2,
    stratify=df['profile_match'],
    random_state=42  # for reproducibility
)

In [117]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"profile_match": "' + str(row['profile_match']) + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['combined_text']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


In [118]:
gpt_train_data = convert_to_gpt35_format(train_data)
gpt_val_data = convert_to_gpt35_format(val_data)

In [119]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(gpt_train_data, training_file_name)
write_to_jsonl(gpt_val_data, validation_file_name)

In [74]:
# Upload Training and Validation Files
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

# Create Fine-Tuning Job
suffix_name = "gpt_conv_ai"
response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
     hyperparameters={
    "n_epochs":2
  }
)
response 

FineTuningJob(id='ftjob-Q8oePaIOhuZtKkcUcxS1He2b', created_at=1709663435, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-JJq0vmR96TJvz4sOKvRr7ht1', result_files=[], status='validating_files', trained_tokens=None, training_file='file-AU5FuMbFdq7qLy3tIuvYlAQk', validation_file='file-jJQ7MChRPdn4H09Myhxmnqho', user_provided_suffix='gpt_conv_ai')

In [120]:
client.fine_tuning.jobs.list(limit=10) # search for suffix_name

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Q8oePaIOhuZtKkcUcxS1He2b', created_at=1709663435, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:gpt-conv-ai:8zU14FGH', finished_at=1709664081, hyperparameters=Hyperparameters(n_epochs=2, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-JJq0vmR96TJvz4sOKvRr7ht1', result_files=['file-a5ZXtVWj0j185mQMT2xpJFHt'], status='succeeded', trained_tokens=37886, training_file='file-AU5FuMbFdq7qLy3tIuvYlAQk', validation_file='file-jJQ7MChRPdn4H09Myhxmnqho', user_provided_suffix='gpt_conv_ai'), FineTuningJob(id='ftjob-7IjxoS60HP3LiB6Il0x28OhM', created_at=1709662898, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:gpt-empathi:8zTrwilN', finished_at=1709663515, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), m

In [121]:
response = client.fine_tuning.jobs.retrieve("ftjob-Q8oePaIOhuZtKkcUcxS1He2b") # retrieve based on yours job
response
     

FineTuningJob(id='ftjob-Q8oePaIOhuZtKkcUcxS1He2b', created_at=1709663435, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:gpt-conv-ai:8zU14FGH', finished_at=1709664081, hyperparameters=Hyperparameters(n_epochs=2, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-JJq0vmR96TJvz4sOKvRr7ht1', result_files=['file-a5ZXtVWj0j185mQMT2xpJFHt'], status='succeeded', trained_tokens=37886, training_file='file-AU5FuMbFdq7qLy3tIuvYlAQk', validation_file='file-jJQ7MChRPdn4H09Myhxmnqho', user_provided_suffix='gpt_conv_ai')

In [122]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:gpt-conv-ai:8zU14FGH


In [131]:
def format_test(row):
    formatted_message = [{"role": "user", "content": row['combined_text']}]
    return formatted_message

def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )
    return response.choices[0].message.content

def store_predictions(test_df,y_test, fine_tuned_model_id):
    test_df['Prediction'] = None
    pred = []
    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        pred.append(prediction_result)
    pred = [int(json.loads(i.replace("'", "\""))['profile_match']) for i in pred]
    print("F1 Score : ", f1_score(pred,y_test,average='weighted'))
    print("Confusion Matrix  : ", confusion_matrix(pred,y_test))
    # test_df.to_csv("predictions.csv")

In [132]:
store_predictions(val_data,val_data['profile_match'], fine_tuned_model_id)

F1 Score :  0.7354838709677419
Confusion Matrix  :  [[ 0  0  0  0]
 [ 0  0  0  0]
 [ 1  6 12  0]
 [ 1  0  0  0]]
